In [2]:
import org.apache.spark.sql.functions._

In [3]:
val conflicts = spark.read.parquet("/tmp/awight/conflicts").cache()
val exits = spark.read.parquet("/tmp/awight/exits").cache()
val linked_exits = spark.read.parquet("/tmp/awight/linked_exits").cache()

conflicts = [conflict_timestamp: timestamp, wiki: string ... 11 more fields]
exits = [exit_timestamp: timestamp, start_time_ts_ms: bigint ... 6 more fields]
linked_exits = [conflict_timestamp: timestamp, wiki: string ... 14 more fields]


[conflict_timestamp: timestamp, wiki: string ... 14 more fields]

In [4]:
val exit_counts = linked_exits.groupBy("exit_action").count()
exit_counts
  .groupBy("exit_action")
  .agg(sum("count").alias("count"))
  .withColumn("percent", round(lit(100) * $"count" /  sum("count").over(), 1))
  .show()

+-----------+-----+-------+
|exit_action|count|percent|
+-----------+-----+-------+
|     cancel|   36|    2.2|
|    unknown|  888|   53.3|
|       save|  742|   44.5|
+-----------+-----+-------+



exit_counts = [exit_action: string, count: bigint]


[exit_action: string, count: bigint]

In [5]:
val with_chunks = linked_exits.filter($"conflict_chunks" > 0)
with_chunks.describe("user_editcount", "conflict_chunks", "conflict_chars").show()
with_chunks
  .groupBy("exit_action")
  .agg(
    count($"*").alias("count"),
    round(mean($"conflict_chunks"), 1).alias("mean_overlapping_chunks"),
    round(stddev($"conflict_chunks"), 1).alias("std_overlapping_chunks"),
    round(mean($"conflict_chars"), 1).alias("mean_overlapping_chars"),
    round(stddev($"conflict_chars"), 1).alias("std_overlapping_chars")
  )
  .show()

+-------+-----------------+------------------+------------------+
|summary|   user_editcount|   conflict_chunks|    conflict_chars|
+-------+-----------------+------------------+------------------+
|  count|              443|               443|               443|
|   mean|27603.18284424379|1.5914221218961626| 964.8081264108353|
| stddev|55763.25081415555|2.1213456222435263|1452.0718079693656|
|    min|                0|                 1|                 0|
|    max|           519415|                25|             14600|
+-------+-----------------+------------------+------------------+

+-----------+-----+-----------------------+----------------------+----------------------+---------------------+
|exit_action|count|mean_overlapping_chunks|std_overlapping_chunks|mean_overlapping_chars|std_overlapping_chars|
+-----------+-----+-----------------------+----------------------+----------------------+---------------------+
|     cancel|   19|                    1.6|                   2.5|   

with_chunks = [conflict_timestamp: timestamp, wiki: string ... 14 more fields]


[conflict_timestamp: timestamp, wiki: string ... 14 more fields]

In [6]:
linked_exits
  .filter($"exit_timestamp".isNotNull)
  .groupBy("exit_action")
  .agg(
    round(mean($"elapsed_s")).alias("mean_elapsed_s"),
    round(stddev($"elapsed_s")).alias("std_elapsed_s"),
    round(min($"elapsed_s")).alias("min_elapsed_s"),
    round(max($"elapsed_s")).alias("max_elapsed_s")
  ).show()

+-----------+--------------+-------------+-------------+-------------+
|exit_action|mean_elapsed_s|std_elapsed_s|min_elapsed_s|max_elapsed_s|
+-----------+--------------+-------------+-------------+-------------+
|     cancel|         139.0|        499.0|          -80|         3016|
|    unknown|          98.0|        706.0|         -211|        16267|
|       save|          86.0|        953.0|        -2876|        25658|
+-----------+--------------+-------------+-------------+-------------+

